In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN # Using DBSCAN
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from scipy.optimize import linear_sum_assignment
import zipfile
import io
import os

# --- Image Processing Functions ---

def load_and_preprocess_image(img_raw: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Converts a raw BGR image to grayscale, applies Gaussian blur, and then
    performs histogram equalization (CLAHE).
    Returns the original BGR, grayscale, and preprocessed grayscale images.
    """
    if img_raw is None:
        print("Error: Input image is None during load/preprocess.")
        return None, None, None

    original_bgr = img_raw.copy()
    gray = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply CLAHE from your K-Means example for consistency in preprocessing
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    equalized = clahe.apply(blurred) 

    return original_bgr, gray, equalized

def segment_image_dbscan(image: np.ndarray, eps: float = 5, min_samples: int = 38) -> tuple[np.ndarray, np.ndarray]:
    """
    Segments an image using DBSCAN clustering.
    Returns:
      - labels: (H*W,) array of cluster assignments per pixel (-1 for noise)
      - segmented_image: (H, W) image where each pixel is colored based on its cluster,
                         with the largest non-noise cluster highlighted (e.g., black)
    """
    rows, cols = image.shape
    
    # Create feature vectors (row, col, intensity) for DBSCAN
    features = []
    for r in range(rows):
        for c in range(cols):
            features.append([r, c, image[r, c]]) # Use preprocessed intensity
    features = np.array(features, dtype=np.float32)

    # Apply DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(features) # labels will contain -1 for noise

    # Create the segmented image based on DBSCAN results
    segmented_image = np.ones((rows, cols, 3), dtype=np.uint8) * 255 # Start with white background

    unique_labels, counts = np.unique(labels, return_counts=True)
    
    # Identify the largest non-noise cluster to highlight (e.g., in black)
    # This helps visualize the primary segmentation result.
    non_noise_labels_indices = np.where(unique_labels != -1)[0]
    
    if len(non_noise_labels_indices) > 0:
        # Get counts for non-noise labels
        non_noise_counts = counts[non_noise_labels_indices]
        # Find the label of the largest non-noise cluster
        largest_cluster_label = unique_labels[non_noise_labels_indices[np.argmax(non_noise_counts)]]
        
        # Color the largest cluster black
        mask = (labels.reshape(rows, cols) == largest_cluster_label)
        segmented_image[mask] = [0, 0, 0] # Set to black (0,0,0)

        # You could also color other non-noise clusters with different shades of gray/colors
        # For simplicity, we highlight the largest cluster and leave others white/noise.
        
    return labels, segmented_image

def blend_images(original_preprocessed_gray: np.ndarray, segmented_dbscan: np.ndarray, alpha: float = 0.7) -> np.ndarray:
    """
    Blends the preprocessed grayscale image with the DBSCAN segmented image.
    Note: DBSCAN segmented image here is expected to be BGR (0 for segmented, 255 for background)
    """
    orig_f = original_preprocessed_gray.astype(np.float32) / 255.0 # Normalize to 0-1
    
    # Convert DBSCAN segmented image to a single channel (0-1 range) for blending
    # Assuming black (0) for foreground and white (255) for background
    segmented_f = cv2.cvtColor(segmented_dbscan, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0

    # Invert the segmented image so that the black region (segmentation) becomes high intensity (1)
    # and white (background) becomes low intensity (0). This makes blending more intuitive.
    inverted_segmented_f = 1.0 - segmented_f

    # Blend: Alpha determines how much of the inverted segmented image is used.
    # Higher alpha means the segmented region will show up more prominently.
    blended_f = (1 - alpha) * orig_f + alpha * inverted_segmented_f
    
    blended_uint8 = cv2.normalize(blended_f, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    return blended_uint8


def enhance_edges(image: np.ndarray) -> np.ndarray:
    """
    Applies a sharpening kernel to enhance edges.
    """
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def compute_segmentation_metrics(
    gt_mask: np.ndarray,
    pred_labels: np.ndarray,
    n_unique_pred_labels: int):
    """
    Compute Accuracy, Normalized Mutual Information (NMI), and Adjusted Rand Index (ARI)
    between a ground-truth mask and DBSCAN cluster labels.
    
    NOTE: For DBSCAN, the 'n_unique_pred_labels' should include the noise label (-1)
    if you want it considered in the confusion matrix. If you want to evaluate only
    the found clusters, you'd filter pred_labels to exclude -1.
    For consistency with the K-Means approach, we'll try to map all unique labels.
    
    Args:
      - gt_mask: 2D array of integer labels (shape: H×W). Each distinct integer is a separate label.
      - pred_labels: 1D array of cluster assignments (length H*W), integers (can include -1 for noise).
      - n_unique_pred_labels: total number of unique labels in pred_labels (including -1 if present).
        
    Returns:
      (accuracy, nmi, ari)
      - accuracy: float in [0,1], computed by finding the optimal one-to-one mapping
                  between cluster IDs and true labels via the Hungarian algorithm.
      - nmi: float in [0,1], normalized mutual information between gt and pred (ignoring label permutations).
      - ari: float in [−1,1], adjusted Rand index between gt and pred.
    """
    gt_flat = gt_mask.flatten()
    pred = pred_labels.copy()

    # Determine unique true labels and remap them to indices [0..n_true-1]
    true_labels_unique = np.unique(gt_flat)
    n_true = len(true_labels_unique)
    label_to_index = {lab: idx for idx, lab in enumerate(true_labels_unique)}
    gt_indices = np.vectorize(label_to_index.get)(gt_flat)  # shape (H*W,)

    # Build confusion matrix: counts[i,j] = #pixels where pred==i and true==j
    # Need to handle potential -1 in pred_labels by mapping them to an index
    pred_labels_unique = np.unique(pred)
    pred_label_to_matrix_idx = {lab: idx for idx, lab in enumerate(pred_labels_unique)}
    pred_matrix_indices = np.vectorize(pred_label_to_matrix_idx.get)(pred)

    counts = np.zeros((len(pred_labels_unique), n_true), dtype=np.int64)
    for idx in range(pred.shape[0]):
        counts[pred_matrix_indices[idx], gt_indices[idx]] += 1

    # Use Hungarian algorithm to find optimal one-to-one assignment between clusters and true labels
    # If the number of predicted labels is different from true labels, linear_sum_assignment handles this by padding.
    cost_matrix = -counts
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # Build mapping: pred_label_value → true_label_value
    # Use pred_labels_unique to map back to original DBSCAN labels (-1, 0, 1, ...)
    mapping = {pred_labels_unique[cluster_matrix_idx]: true_labels_unique[true_idx] for cluster_matrix_idx, true_idx in zip(row_ind, col_ind)}
    
    # Map predicted labels using the optimal mapping
    pred_mapped = np.array([mapping.get(label, -1) for label in pred]) # Default to -1 if label not mapped (e.g., unmapped noise)

    # Compute Accuracy
    # Filter out ground truth labels that don't have a corresponding cluster if necessary,
    # or ensure gt_mask is consistent with desired comparison for accuracy.
    # For now, we'll compare everything, including noise.
    acc = accuracy_score(gt_flat, pred_mapped)

    # Compute NMI and ARI on the original labels (they are permutation-invariant internally)
    nmi = normalized_mutual_info_score(gt_flat, pred)
    ari = adjusted_rand_score(gt_flat, pred)

    return acc, nmi, ari

def plot_metric_barchart(metrics_dict: dict, title: str = 'Segmentation Metrics'):
    """
    Given a dictionary with keys ['Accuracy','NMI','ARI'] and values in [0,1],
    plot a bar chart showing these three metrics.
    """
    labels = list(metrics_dict.keys())
    values = [metrics_dict[k] for k in labels]

    plt.figure(figsize=(7, 5))
    bars = plt.bar(labels, values, alpha=0.8, edgecolor='black', color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.ylim(0, 1.0)
    plt.ylabel('Score')
    plt.title(title)
    for bar, v in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width() / 2, v + 0.02, f"{v:.2f}", ha='center', va='bottom')
    plt.tight_layout()
    plt.show()

def plot_intensity_histogram_with_centroids(image: np.ndarray, title: str, ax: plt.Axes):
    """
    Plot histogram of pixel intensities (0..255) of the given image,
    and overlay a dashed vertical line at the mean intensity (centroid).
    """
    if len(image.shape) == 3: # If BGR, convert to grayscale for histogram
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image_gray = image

    # Calculate histogram
    hist = cv2.calcHist([image_gray], [0], None, [256], [0, 256])
    ax.plot(hist, color='black')
    ax.set_title(title)
    ax.set_xlabel('Pixel Intensity')
    ax.set_ylabel('Frequency')

    # Calculate and plot centroid (mean intensity value)
    flat_image = image_gray.flatten()
    if len(flat_image) > 0:
        centroid = np.mean(flat_image)
        ax.axvline(centroid, color='red', linestyle='dashed', linewidth=1)
        ax.text(centroid + 5, ax.get_ylim()[1] * 0.9, f'Centroid: {centroid:.2f}', color='red')


def show_results(orig: np.ndarray, clahe_img: np.ndarray,
                 dbscan_segmented: np.ndarray, final_enhanced: np.ndarray,
                 image_title: str):
    """
    Display four images side by side with titles:
    [Original | CLAHE Preprocessed | DBSCAN Segmented | Final Enhanced].
    """
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    titles = [f'Original\n({image_title})', 'CLAHE Preprocessed', 'DBSCAN Segmented', 'Final Enhanced']
    # Ensure all images are BGR for consistent imshow display (Matplotlib expects RGB)
    orig_rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
    clahe_rgb = cv2.cvtColor(clahe_img, cv2.COLOR_GRAY2RGB) # Convert grayscale to RGB for display
    dbscan_rgb = cv2.cvtColor(dbscan_segmented, cv2.COLOR_BGR2RGB) # Already BGR from segment_image_dbscan
    final_rgb = cv2.cvtColor(final_enhanced, cv2.COLOR_GRAY2RGB) # Convert final enhanced (grayscale) to RGB
    
    imgs   = [orig_rgb, clahe_rgb, dbscan_rgb, final_rgb]

    for ax, img, title in zip(axes, imgs, titles):
        ax.imshow(img) # Matplotlib expects RGB
        ax.set_title(title)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# --- Main Execution Block for Jupyter Notebook (Local) ---

print("Welcome! This script will process images from a zip file using DBSCAN.")
zip_file_path = input("Please enter the full path to your zip file (e.g., C:\\Users\\YourName\\images.zip): ").strip()

# Handle potential quotes from drag-and-drop on Windows
if zip_file_path.startswith("'") and zip_file_path.endswith("'"):
    zip_file_path = zip_file_path[1:-1]
elif zip_file_path.startswith('"') and zip_file_path.endswith('"'):
    zip_file_path = zip_file_path[1:-1]

processed_image_data = [] # Stores (original_bgr, final_enhanced_gray, preprocessed_gray, dbscan_segmented_bgr, dbscan_labels_flat)
image_names = []
all_metrics_dict = {} # Stores metrics for each image: {'Image1': {'Acc': ..., 'NMI': ..., 'ARI': ...}, ...}

# Create a directory to save enhanced images
output_dir = "enhanced_dbscan_output"
os.makedirs(output_dir, exist_ok=True)

# --- Important: Placeholder for Ground Truth ---
# For demonstration, we'll assume the original grayscale image itself is a simple "mask"
# or that you will provide a specific ground truth image within the zip for comparison.
# If you have actual masked training images, place them in the zip and adjust the loading logic.
# For now, we'll use a simplified assumption for comparison if a GT is not explicitly found.

try:
    if not os.path.exists(zip_file_path):
        raise FileNotFoundError(f"Zip file not found at: {zip_file_path}")

    with zipfile.ZipFile(zip_file_path, 'r') as zf:
        image_files = [f for f in zf.namelist() if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff'))]
        
        if not image_files:
            print("No image files found in the zip archive.")
        else:
            print(f"Found {len(image_files)} image(s) in the zip file.")
            
            # Process up to 5 images
            images_to_process = image_files[:5]
            if len(image_files) > 5:
                print(f"Note: Processing only the first {len(images_to_process)} images found in the zip file.")

            for img_name in images_to_process:
                print(f"\nProcessing {img_name}...")
                with zf.open(img_name) as img_file:
                    img_data = img_file.read()
                    img_array = np.frombuffer(img_data, np.uint8)
                    
                    img_raw = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

                    if img_raw is None:
                        print(f"Warning: Could not decode image {img_name}. Skipping.")
                        continue

                    # --- DBSCAN specific processing steps ---
                    original_bgr, original_gray, preprocessed_gray = load_and_preprocess_image(img_raw)
                    
                    if preprocessed_gray is None:
                        print(f"Skipping {img_name} due to preprocessing error.")
                        continue

                    # DBSCAN parameters (adjust these based on your image characteristics!)
                    # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
                    # min_samples: The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
                    dbscan_labels, dbscan_segmented_bgr = segment_image_dbscan(preprocessed_gray, eps=5, min_samples=38) # Default values from your previous code
                    
                    # Blend the preprocessed image with the segmented result
                    blended_image_gray = blend_images(preprocessed_gray, dbscan_segmented_bgr, alpha=0.7)
                    
                    # Apply final edge enhancement
                    final_enhanced_image_gray = enhance_edges(blended_image_gray)
                    
                    processed_image_data.append((original_bgr, final_enhanced_image_gray, original_gray, dbscan_segmented_bgr, dbscan_labels))
                    image_names.append(img_name)
                    
                    # --- Metrics Calculation (requires Ground Truth) ---
                    # IMPORTANT: For real metric comparison, you need actual ground truth masks.
                    # As a placeholder, we will use the preprocessed image as a pseudo ground truth
                    # or assume the GT mask is in the zip with a specific naming convention (e.g., '_mask.png').
                    # For this example, we'll simulate a simple 2-label GT based on intensity.
                    # If you have specific GT masks, please provide them in the zip!
                    
                    # Pseudo Ground Truth (replace with actual GT loading if available)
                    # For a simple binary GT, you might threshold the original image
                    # Here, let's create a simple binary "mask" from preprocessed_gray
                    # by just thresholding it. This is NOT a real GT, but allows the metrics functions to run.
                    # You'd load a dedicated `gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)` for real analysis.
                    _, pseudo_gt_mask = cv2.threshold(preprocessed_gray, np.mean(preprocessed_gray), 1, cv2.THRESH_BINARY)
                    pseudo_gt_mask = pseudo_gt_mask.astype(np.int32) # Ensure integer labels
                    
                    # Compute metrics
                    # n_unique_pred_labels must include -1 if it's in dbscan_labels
                    acc, nmi, ari = compute_segmentation_metrics(
                        gt_mask=pseudo_gt_mask, 
                        pred_labels=dbscan_labels, 
                        n_unique_pred_labels=len(np.unique(dbscan_labels))
                    )
                    
                    current_metrics = {'Accuracy': acc, 'NMI': nmi, 'ARI': ari}
                    all_metrics_dict[img_name] = current_metrics
                    print(f"  Metrics for {img_name}: Accuracy={acc:.4f}, NMI={nmi:.4f}, ARI={ari:.4f}")
                    
                    # Save the enhanced image
                    base_name = os.path.splitext(os.path.basename(img_name))[0]
                    output_path = os.path.join(output_dir, f"enhanced_dbscan_{base_name}.png")
                    cv2.imwrite(output_path, cv2.cvtColor(final_enhanced_image_gray, cv2.COLOR_GRAY2BGR)) # Save as BGR
                    print(f"Enhanced image saved to {output_path}")

except FileNotFoundError as e:
    print(f"Error: {e}")
except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file. Please check your file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# --- Display Results in Jupyter Notebook ---

if not processed_image_data:
    print("No images were successfully processed to display results.")
else:
    # 1. Display all enhanced images (Original vs. CLAHE vs. DBSCAN Segmented vs. Final Enhanced)
    print("\n--- Original vs. DBSCAN Enhanced Images ---")
    for i, (original_bgr, final_enhanced_gray, preprocessed_gray, dbscan_segmented_bgr, _) in enumerate(processed_image_data):
        show_results(original_bgr, preprocessed_gray, dbscan_segmented_bgr, final_enhanced_gray, os.path.basename(image_names[i]))

    # 2. Bar chart of Segmentation Metrics for each image
    print("\n--- Segmentation Metrics for Each Image (DBSCAN) ---")
    for img_name, metrics in all_metrics_dict.items():
        print(f"\nMetrics for {os.path.basename(img_name)}:")
        plot_metric_barchart(metrics, title=f'DBSCAN Metrics for {os.path.basename(img_name)}')

    # 3. Histograms with Centroids for Original Grayscale and Final Enhanced Images
    print("\n--- Histograms with Centroids (Original vs. Final DBSCAN Enhanced) ---")
    num_images = len(processed_image_data)
    fig_histograms, axes_hist = plt.subplots(num_images, 2, figsize=(15, 5 * num_images))
    
    if num_images == 1:
        axes_hist = np.array([axes_hist]) # Ensure axes_hist is 2D for consistent indexing

    for i, (original_bgr, final_enhanced_gray, original_gray, _, _) in enumerate(processed_image_data):
        # Histogram for Original Grayscale Image
        plot_histogram_with_centroids(original_gray, f'Original Histogram: {os.path.basename(image_names[i])}', axes_hist[i, 0])
        
        # Histogram for Final Enhanced Image (after DBSCAN, blending, and sharpening)
        plot_histogram_with_centroids(final_enhanced_image_gray, f'DBSCAN Final Enhanced Histogram: {os.path.basename(image_names[i])}', axes_hist[i, 1])

    plt.tight_layout()
    plt.show()

Welcome! This script will process images from a zip file using DBSCAN.
